In [1]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

In [2]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

In [4]:
import requests

url = "https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf"
response = requests.get(url)
with open("gemini.pdf", "wb") as file:
    file.write(response.content)

document_file = client.files.upload(file='gemini.pdf')

In [5]:
request = 'Tell me about the training process used here.'

def summarize_doc(request: str) -> str:
    config = types.GenerateContentConfig(temperature=0.0)
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=config,
        contents=[request, document_file])
    return response.text

summary = summarize_doc(request)
Markdown(summary)

Certainly! Let's break down the training process used for Gemini 1.5 Pro, based on the information provided in the document.

**Key Aspects of the Training Process:**

1.  **Model Architecture:**
    *   Gemini 1.5 Pro is a sparse Mixture-of-Experts (MoE) Transformer-based model.
    *   It builds upon the research advances of Gemini 1.0 and a longer history of MoE research at Google.
    *   MoE models use a learned routing function to direct inputs to a subset of the model's parameters for processing. This allows for a larger total parameter count while keeping the number of activated parameters constant.

2.  **Training Infrastructure:**
    *   Trained on multiple 4096-chip pods of Google's TPUv4 accelerators.
    *   The training is distributed across multiple datacenters.

3.  **Training Data:**
    *   A variety of multimodal and multilingual data is used.
    *   The pre-training dataset includes data sourced from various domains, including web documents, code, images, audio, and video content.

4.  **Instruction Tuning:**
    *   Gemini 1.5 Pro is fine-tuned on a collection of multimodal data containing paired instructions and appropriate responses.
    *   Further tuning is based on human preference data.

5.  **Long-Context Understanding:**
    *   The model incorporates significant architecture changes to enable long-context understanding of inputs up to 10 million tokens without degrading performance.

6.  **Safety Mitigations:**
    *   Supervised fine-tuning (SFT) and reinforcement learning through human feedback (RLHF) are used to mitigate safety risks.
    *   The safety mitigation is focused on adversarial or "harm-inducing" queries.
    *   New image-to-text SFT data is incorporated, as safety SFT data for text-only queries was not as effective for harm-inducing image-to-text queries.

**In summary:** The training process for Gemini 1.5 Pro involves a combination of a sophisticated MoE Transformer architecture, massive computational resources, diverse multimodal data, instruction tuning, and safety mitigations. The model is designed to handle extremely long contexts, enabling it to process and reason over large amounts of information from various modalities.

In [6]:
import enum

# Define the evaluation prompt
SUMMARY_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your rating, and only choose ratings from the Rating Rubric.

# Evaluation
## Metric Definition
You will be assessing summarization quality, which measures the overall ability to summarize text. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a summarization task and the context to be summarized are provided in the user prompt. The response should be shorter than the text in the context. The response should not contain information that is not present in the context.

## Criteria
Instruction following: The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context. The response does not reference any outside information.
Conciseness: The response summarizes the relevant details in the original text without a significant loss in key information without being too verbose or terse.
Fluency: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The summary follows instructions, is grounded, is concise, and fluent.
4: (Good). The summary follows instructions, is grounded, concise, and fluent.
3: (Ok). The summary mostly follows instructions, is grounded, but is not very concise and is not fluent.
2: (Bad). The summary is grounded, but does not follow the instructions.
1: (Very bad). The summary is not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs

### Prompt
{prompt}

## AI-generated Response
{response}
"""

class SummaryRating(enum.Enum):
    """Enum for rating the summary."""
    VERY_GOOD = "5"
    GOOD = "4"
    OK = "3"
    BAD = "2"
    VERY_BAD = "1"
    
def eval_summary(prompt, ai_response):
    chat = client.chats.create(model="gemini-2.0-flash")
    response = chat.send_message(
        message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
    )
    
    verbose_eval = response.text
    
    # Extract the rating from the verbose evaluation
    structured_output_config = types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=SummaryRating)
    
    response = chat.send_message(
        message="Convert the final score.",
        config=structured_output_config)
    
    structured_eval = response.parsed
    
    return verbose_eval, structured_eval

text_eval, structured_eval = eval_summary(prompt=[request, document_file], ai_response=summary)
Markdown(text_eval)

## Evaluation Explanation

STEP 1:
The response summarizes the training process of Gemini 1.5 Pro. It breaks down the response into key aspects of the training process, including model architecture, training infrastructure, training data, instruction tuning, long-context understanding, and safety mitigations. The response provides a clear and organized summary of the training process.

STEP 2:
Instruction Following: The response accurately summarizes the key aspects of the training process, following the instructions provided in the prompt.
Groundedness: The response is grounded in the information contained in the document and does not reference any outside information.
Conciseness: The response effectively summarizes the relevant details of the training process without being too verbose or terse.
Fluency: The response is well-organized and easy to read, with clear headings and bullet points.

The response is well-written, organized, and informative, providing a comprehensive summary of the training process used for Gemini 1.5 Pro based on the information in the document. Overall, the response demonstrates a strong understanding of the material and effectively communicates the key aspects of the training process. Therefore, I rate it as very good.

## Rating
5


In [7]:
structured_eval

<SummaryRating.VERY_GOOD: '5'>

In [8]:
new_prompt = "Explain like I'm 5 the training process"

def run_and_eval_summary(prompt):
  """Generate and evaluate the summary using the new prompt."""
  summary = summarize_doc(new_prompt)
  display(Markdown(summary + '\n-----'))

  text, struct = eval_summary([new_prompt, document_file], summary)
  display(Markdown(text + '\n-----'))
  print(struct)

run_and_eval_summary(new_prompt)

Okay, I can explain the training process of a large language model like Gemini 1.5 Pro in a way that a 5-year-old can understand.

Imagine you have a puppy, and you want to teach it to understand and follow your instructions.

1.  **Lots and Lots of Examples:** First, you show the puppy many, many examples of things you want it to learn. For example, you show it pictures of cats and say "cat," pictures of dogs and say "dog," and so on. You also read it lots of stories and talk to it all the time. The puppy is like the computer model, and all the pictures, stories, and conversations are the training data.

2.  **Learning the Patterns:** The puppy starts to notice patterns. It sees that cats have pointy ears and whiskers, and dogs have floppy ears and wagging tails. It also learns that certain words go together, like "good" and "boy." The computer model does the same thing. It looks for patterns in the training data and learns how words, images, and sounds are related.

3.  **Making Predictions:** Now, you show the puppy a new picture, and you ask, "What's this?" The puppy tries to guess based on what it has learned. If it says "cat" when it sees a cat, that's good! If it says "dog," you gently correct it. The computer model also makes predictions. It might try to guess the next word in a sentence or answer a question about a picture.

4.  **Getting Better and Better:** Every time the puppy makes a mistake, you help it learn from that mistake. You give it feedback so it can do better next time. The computer model also gets feedback. It adjusts its internal settings to make better predictions in the future. This process is repeated over and over again, with more and more examples, until the puppy (or the computer model) becomes very good at understanding and following instructions.

So, in short, training a large language model is like teaching a puppy: you show it lots of examples, it learns the patterns, it makes predictions, and it gets better and better with feedback. The more examples you give it, the smarter it becomes!
-----

## Evaluation
STEP 1: The response uses an analogy to explain the training process. The analogy is a puppy learning from feedback. This response doesn't follow the instructions. I'm rating it a 1.

STEP 2: Based on the evaluation, I choose a rating of 1.

## Rating:
1
-----

SummaryRating.VERY_BAD


In [9]:
# Pointwise evaluation
import functools

# Try these instructions, or edit and add your own.
terse_guidance = "Answer the following question in a single sentence, or as close to that as possible."
moderate_guidance = "Provide a brief answer to the following question, use a citation if necessary, but only enough to answer the question."
cited_guidance = "Provide a thorough, detailed answer to the following question, citing the document and supplying additional background information as much as possible."
guidance_options = {
    'Terse': terse_guidance,
    'Moderate': moderate_guidance,
    'Cited': cited_guidance,
}

questions = [
    # Un-comment one or more questions to try here, or add your own.
    # Evaluating more questions will take more time, but produces results
    # with higher confidence. In a production system, you may have hundreds
    # of questions to evaluate a complex system.

    # "What metric(s) are used to evaluate long context performance?",
    "How does the model perform on code tasks?",
    "How many layers does it have?",
    # "Why is it called Gemini?",
]

if not questions:
  raise NotImplementedError('Add some questions to evaluate!')

@functools.cache
def answer_question(question: str, guidance: str = '') -> str:
  """Generate an answer to the question using the uploaded document and guidance."""
  config = types.GenerateContentConfig(
      temperature=0.0,
      system_instruction=guidance,
  )
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[question, document_file],
  )

  return response.text


answer = answer_question(questions[0], terse_guidance)
Markdown(answer)

Gemini 1.5 Pro performs well on code tasks, improving over previous Gemini models and approaching state-of-the-art performance.


In [10]:
# set up a question-answering evaluator
import enum

QA_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user prompt and an AI-generated responses.
You should first read the user prompt carefully for analyzing the task, and then evaluate the quality of the responses based on and rules provided in the Evaluation section below.

# Evaluation
## Metric Definition
You will be assessing question answering quality, which measures the overall quality of the answer to the question in the user prompt. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a question-answering task is provided in the user prompt. The response should not contain information that is not present in the context (if it is provided).

You will assign the writing response a score from 5, 4, 3, 2, 1, following the Rating Rubric and Evaluation Steps.
Give step-by-step explanations for your scoring, and only choose scores from 5, 4, 3, 2, 1.

## Criteria Definition
Instruction following: The response demonstrates a clear understanding of the question answering task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context if the context is present in the user prompt. The response does not reference any outside information.
Completeness: The response completely answers the question with sufficient detail.
Fluent: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The answer follows instructions, is grounded, complete, and fluent.
4: (Good). The answer follows instructions, is grounded, complete, but is not very fluent.
3: (Ok). The answer mostly follows instructions, is grounded, answers the question partially and is not very fluent.
2: (Bad). The answer does not follow the instructions very well, is incomplete or not fully grounded.
1: (Very bad). The answer does not follow the instructions, is wrong and not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness,completeness, and fluency according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs
### Prompt
{prompt}

## AI-generated Response
{response}
"""

class AnswerRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'

@functools.cache
def eval_answer(prompt, ai_response, n=1):
  """Evaluate the generated answer against the prompt/question used."""
  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=QA_PROMPT.format(prompt=[prompt, document_file], response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=AnswerRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_answer(prompt=questions[0], ai_response=answer)
display(Markdown(text_eval))
print(struct_eval)

STEP 1:
The response is grounded since it only uses information that is present in the document. The response is fluent, complete and follows all instructions.

STEP 2:
I would rate this response a 5 because it is a very good answer.

AnswerRating.VERY_GOOD


In [11]:
# run in a loop
import collections
import itertools

# Number of times to repeat each task in order to reduce error and calculate an average.
# Increasing it will take longer but give better results, try 2 or 3 to start.
NUM_ITERATIONS = 3

scores = collections.defaultdict(int)
responses = collections.defaultdict(list)

for question in questions:
  display(Markdown(f'## {question}'))
  for guidance, guide_prompt in guidance_options.items():

    for n in range(NUM_ITERATIONS):
      # Generate a response.
      answer = answer_question(question, guide_prompt)

      # Evaluate the response (note that the guidance prompt is not passed).
      written_eval, struct_eval = eval_answer(question, answer, n)
      print(f'{guidance}: {struct_eval}')

      # Save the numeric score.
      scores[guidance] += int(struct_eval.value)

      # Save the responses, in case you wish to inspect them.
      responses[(guidance, question)].append((answer, written_eval))

## How does the model perform on code tasks?

Terse: AnswerRating.VERY_GOOD
Terse: AnswerRating.VERY_GOOD
Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD


## How many layers does it have?

Terse: AnswerRating.VERY_GOOD
Terse: AnswerRating.VERY_GOOD
Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_BAD
Moderate: AnswerRating.BAD
Moderate: AnswerRating.OK
Cited: AnswerRating.VERY_GOOD
Cited: AnswerRating.OK
Cited: AnswerRating.BAD


In [12]:
for guidance, score in scores.items():
  # Average the scores.
  avg_score = score / (NUM_ITERATIONS * len(questions))
  nearest = AnswerRating(str(round(avg_score)))
  print(f'{guidance}: {avg_score:.2f} ({nearest})')

Terse: 5.00 (AnswerRating.VERY_GOOD)
Moderate: 3.50 (AnswerRating.GOOD)
Cited: 4.17 (AnswerRating.GOOD)
